In [1]:
from numpy import linalg as LA
import numpy as np
import pandas as pd
import math
import networkx as nx
import community


def compute_C_minus_C0(lambdas,v,lambda_plus):
    N=len(lambdas)
    C_clean=np.zeros((N, N))
    
    v_m=np.matrix(v)
    # _s stands for _structure below
    for i in range(N-1):
        if lambdas[i]>lambda_plus:
            C_clean=C_clean+lambdas[i] * np.dot(v_m[i,].T,v_m[i,]) 
    return C_clean    
    
    
def LouvainCorrelationClustering(R):   # R is a matrix of return
    N=R.shape[1]
    T=R.shape[0]

    q=N*1./T
    lambda_plus=(1.+np.sqrt(q))**2

    C=R.corr()
    lambdas, v = LA.eigh(C)
    
    C_s=compute_C_minus_C0(lambdas,v,lambda_plus)
    C_s=np.abs(C_s)
    
    mygraph= nx.from_numpy_matrix(np.abs(C_s))
    partition = community.community_louvain.best_partition(mygraph)

    DF=pd.DataFrame.from_dict(partition,orient="index")
    return(DF)




In [2]:
from rpy2.robjects.packages import importr
utils = importr('utils')
utils.install_packages("data.table")
 


rpy2.rinterface.NULL

In [3]:
import rpy2.robjects as robjects
r = robjects.r

#importr("data.table")
newfunc=r.source("libClusteringGiadaMarsiliFast.R")
aggregateClusters=robjects.globalenv["aggregateClusters"]


In [4]:
import numpy as np

import feather   # pip install feather-format

DF=feather.read_dataframe("data/clean/us_equities_logreturns.feather")

DF_cut=DF.iloc[12000:14000]

In [5]:
sel=DF_cut.isnull().sum(axis=0)>0
DF_cut=DF_cut.drop(columns=DF_cut.columns[sel])

In [6]:
import rpy2.robjects.numpy2ri
rpy2.robjects.numpy2ri.activate()

C=DF_cut.corr()
myres=aggregateClusters(C.values)

In [7]:
myres


R object with classes: ('list',) mapped to:
<ListVector - Python:0x7fa49c655788 / R:0x55784c6796d8>
[ListV..., StrVe..., ListV..., ListV..., ListV..., ListV...]
R object with classes: ('list',) mapped to:
<ListVector - Python:0x7fa49c655788 / R:0x55784c6796d8>
[ListV..., StrVe..., ListV..., ListV..., ListV..., ListV...]
  pair.merged: <class 'rpy2.robjects.vectors.StrVector'>
  R object with classes: ('character',) mapped to:
<StrVector - Python:0x7fa49c655b88 / R:0x55784aac0c98>
['260', '272']
R object with classes: ('list',) mapped to:
<ListVector - Python:0x7fa49c655788 / R:0x55784c6796d8>
[ListV..., StrVe..., ListV..., ListV..., ListV..., ListV...]
R object with classes: ('list',) mapped to:
<ListVector - Python:0x7fa49c655788 / R:0x55784c6796d8>
[ListV..., StrVe..., ListV..., ListV..., ListV..., ListV...]
R object with classes: ('list',) mapped to:
<ListVector - Python:0x7fa49c655788 / R:0x55784c6796d8>
[ListV..., StrVe..., ListV..., ListV..., ListV..., ListV...]
R object with classes: ('list',) mapped to:
<ListVector - Python:0x7fa49c655788 / R:0x55784c6796d8>
[ListV..., StrVe..., ListV..., ListV..., ListV..., ListV...]

In [10]:
myres[3]   # this is a dictionary (List in the R language) which contains the membership of each cluster, on cluster at a time

## there are 49 clusters (with the chosen parameters), which is way more than Louvain yields when applied onces. 

np.array(myres[3][0]) # first cluster

array([  1,   2,  27,  61, 242,  32,  87, 117,  70,  72, 172, 112, 127,
       132, 217, 207,   6,   7,   9,  55,  18,  17,  20,  80, 257, 151,
        75,  58,  57, 116,  59,  22, 142, 174,  31,  44, 159, 162, 200,
       196,  37,  68, 118, 101, 125, 157, 194,  81, 191, 184, 188,  56,
       183, 212, 124, 136,  93,  48, 100,  91, 119, 141, 206, 248, 103,
        95,  43, 175, 286, 163,  85], dtype=int32)